In [1]:
import sys
import os
sys.path.append("..")
from biokey.data import DataInterface
import biokey.tools
import credentials

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
% matplotlib inline

In [3]:
data = DataInterface(credentials.postgres)

Loading Data
	- Attempting cache load
	- Loaded strokes from cache
Processing Data
	- Attempting cache load
	- Loaded dwell and flight from cache
Done Loading



In [4]:
datasets = data.get_user_sets('1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f')
datasets.train = biokey.tools.filter_keys(datasets.train)
datasets.test = biokey.tools.filter_keys(datasets.test)

# Form Dwell Sequence Frames

In [5]:
def generate_arrays(df):
    downsample_rate = 10
    train = df.copy()[['key_enum', 'down', 'up', 'is_user']]
    train.down = ((df.down - df.down.min())/downsample_rate).round().astype('int64')
    train.up = ((df.up - df.down.min())/downsample_rate).round().astype('int64')
    train.is_user = (train.is_user).astype('int32')
    train['not_user'] = (train.is_user == False).astype('int32')
    
    sample_freq = 100*60
    sample_length = train.up.max()
    # only select first n frames
    n = 100
    sample_length = int(sample_length/sample_freq)*n
    num_rows = int(sample_length/sample_freq)
    num_keys = train.key_enum.unique().max() + 1

    x_series = np.zeros((num_rows*sample_freq , num_keys), dtype='int16')
    y_series = np.zeros((num_rows*sample_freq, 3), dtype='int16')
    y_series[:,2] = 1 # Set inactive to true
    #(is_user, is_imposter, inactive)
    for x in train.itertuples():
        x_series[x.down:x.up, x.key_enum] = 1
        y_series[x.down:x.up, 0] = x.is_user
        y_series[x.down:x.up, 1] = x.not_user
        y_series[x.down:x.up, 2] = 0
    x_batches = np.array(np.array_split(x_series, num_rows))
    y_batches = np.array(np.array_split(y_series, num_rows))
    y_sample = y_batches.sum(axis=1)
    y_sample[:,0] = (y_sample[:,0] > y_sample[:,1]).astype('int16')
    y_sample[:,1] = (y_sample[:,0] < y_sample[:,1]).astype('int16')
    y_sample[:,2] = ((y_sample[:,0] == 0) & (y_sample[:,1] == 0)).astype('int16')
    
    return x_batches, y_sample

In [6]:
X, Y = generate_arrays(datasets.train)

In [7]:
X_test, Y_test = generate_arrays(datasets.test)

# Visualize for Fun

In [8]:
select_index = 1020
select_length = 5
plt.imshow(np.concatenate(X[select_index:select_length+select_index]).T, cmap='hot', interpolation='nearest', aspect='auto')
plt.show()
Y[select_index:select_length+select_index]

ValueError: need at least one array to concatenate

# Use Keras

In [8]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import TensorBoard
from sklearn.metrics import roc_curve, auc
from phased_lstm_keras.PhasedLSTM import PhasedLSTM as PLSTM

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
# Parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 100

# Network Parameters
input_shape = (X.shape[1], X.shape[2])
n_hidden_1 = 32 # 1st layer number of features
n_hidden_2 = 32 # 2nd layer number of features
n_classes = Y.shape[1] # Number of classes to predict

# PhasedLSTM

In [ ]:
n_hidden_1 = 32
# create model
model_PLSTM = Sequential()
model_PLSTM.add(PLSTM(n_hidden_1, input_shape=input_shape, implementation=2))
model_PLSTM.add(Dense(n_classes, activation='softmax'))
# Compile model
model_PLSTM.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_PLSTM.summary()
tbCallBack = TensorBoard(log_dir='./Graph/plstm'+str(n_hidden_1), histogram_freq=0, write_graph=True, write_images=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
phased_lstm_1 (PhasedLSTM)   (None, 32)                7648      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 7,747
Trainable params: 7,747
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_PLSTM.fit(X, Y, epochs=training_epochs, batch_size=batch_size, callbacks=[tbCallBack])
model_PLSTM.save('model_plstm'+str(n_hidden_1)+'.h5')

Epoch 1/5
1404/1404 [==============================] - 192s 137ms/step - loss: 0.9503 - acc: 0.7358
Epoch 2/5
 100/1404 [=>............................] - ETA: 2:44 - loss: 0.7166 - acc: 0.7900

In [ ]:
n_hidden_1 = 256
# create model
model_PLSTM = Sequential()
model_PLSTM.add(PLSTM(n_hidden_1, input_shape=input_shape, implementation=2))
model_PLSTM.add(Dense(n_classes, activation='softmax'))
# Compile model
model_PLSTM.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_PLSTM.summary()
tbCallBack = TensorBoard(log_dir='./Graph/plstm'+str(n_hidden_1), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
model_PLSTM.fit(X, Y, epochs=training_epochs, batch_size=batch_size, callbacks=[tbCallBack])
model_PLSTM.save('model_plstm'+str(n_hidden_1)+'.h5')

# LSTM

In [ ]:
n_hidden_1 = 32
# create model
model_LSTM = Sequential()
model_LSTM.add(PLSTM(n_hidden_1, input_shape=input_shape, implementation=2))
model_LSTM.add(Dense(n_classes, activation='softmax'))
# Compile model
model_LSTM.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_LSTM.summary()
tbCallBack = TensorBoard(log_dir='./Graph/lstm'+str(n_hidden_1), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
model_LSTM.fit(X, Y, epochs=training_epochs, batch_size=batch_size, callbacks=[tbCallBack])
model_PLSTM.save('model_lstm'+str(n_hidden_1)+'.h5')

In [ ]:
n_hidden_1 = 256
# create model
model_LSTM = Sequential()
model_LSTM.add(PLSTM(n_hidden_1, input_shape=input_shape, implementation=2))
model_LSTM.add(Dense(n_classes, activation='softmax'))
# Compile model
model_LSTM.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_LSTM.summary()
tbCallBack = TensorBoard(log_dir='./Graph/lstm'+str(n_hidden_1), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
model_LSTM.fit(X, Y, epochs=training_epochs, batch_size=batch_size, callbacks=[tbCallBack])
model_PLSTM.save('model_lstm'+str(n_hidden_1)+'.h5')

# Assess Results

In [ ]:
def generate_results(y_test, y_score):
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.show()
    print('AUC: %f' % roc_auc)